In [ ]:
import json
import requests
import numpy as np
import pandas as pd

In [ ]:
# pull data from open-meteo api
try:
    # get most recent (current day - 2) hourly data: temp(2m), rel_humid(2m), apparent_temp, uv_index, is_day_or_night
    # location: san francisco
    # timezone: americas/los angeles (gmt - 7/8)
    # BUT store using utc timestamp
    open_meteo_endpoint ='https://historical-forecast-api.open-meteo.com/v1/forecast?latitude=37.7749&longitude=-122.4194&start_date=2025-07-16&end_date=2025-07-16&hourly=temperature_2m,apparent_temperature,uv_index,is_day,relative_humidity_2m&timezone=America%2FLos_Angeles&temperature_unit=fahrenheit'
    response = requests.get(url=open_meteo_endpoint)
    response.raise_for_status()
    response_data = response.json()
except Exception as e:
    print(f'Error: {e}')


In [ ]:
df = pd.DataFrame.from_dict(data=response_data)

df

In [ ]:
df.info()

In [ ]:
# convert the data into an hourly data frame (or timestamped by time)
# each row is a time, and all of the hourly attributes at that time

columns_to_extract = df.index

hourly_data = {
    key: df.at[key, 'hourly']
    for key in columns_to_extract
}

hourly_df = pd.DataFrame(data=hourly_data)
# parse local time into localized datetime
hourly_df['time'] = pd.to_datetime(hourly_df['time'])
hourly_df['time'] = hourly_df['time'].dt.tz_localize('America/Los_Angeles')
# calc utc_time
hourly_df['utc_time'] = hourly_df['time'].dt.tz_convert('UTC')
# set utc_time as index
hourly_df.set_index('utc_time', inplace=True)

hourly_df